In [1]:
import re

import pandas as pd
import numpy as np

### dataディレクトリの中に格納されているxlsxファイルのパスを取得

In [2]:
import glob

file_list = glob.glob('data/enavi*.csv')

file_list

['data\\enavi201904(6875).csv',
 'data\\enavi201905(6875).csv',
 'data\\enavi201906(6875).csv',
 'data\\enavi201907(6875).csv',
 'data\\enavi201908(6875).csv',
 'data\\enavi201909(6875).csv',
 'data\\enavi201910(6875).csv',
 'data\\enavi201911(6875).csv',
 'data\\enavi201912(6875).csv',
 'data\\enavi202001(6875).csv',
 'data\\enavi202002(6875).csv',
 'data\\enavi202003(6875).csv',
 'data\\enavi202004(6875).csv',
 'data\\enavi202005(6875).csv',
 'data\\enavi202006(6875).csv']

### データフレームとして1ファイルのみ読み込み

In [3]:
df = pd.read_csv(file_list[0])

df['test/test'] = 'test'

display(df.shape, df.head())

(55, 11)

,利用日,利用店名・商品名,利用者,支払方法,利用金額,支払手数料,支払総額,4月支払金額,5月繰越残高,新規サイン,test/test
0,2019/03/30,AMAZON.CO.JP,本人,1回払い,1782.0,0.0,1782.0,1782.0,0.0,*,test
1,2019/03/30,ﾏｲﾊﾞｽｹﾂﾄﾒｸﾞﾛﾀｲｺﾊﾞ,本人,1回払い,1164.0,0.0,1164.0,1164.0,0.0,*,test
2,2019/03/29,ﾏｲﾊﾞｽｹﾂﾄﾒｸﾞﾛﾀｲｺﾊﾞ,本人,1回払い,2766.0,0.0,2766.0,2766.0,0.0,*,test
3,2019/03/28,ローソン,本人,1回払い,133.0,0.0,133.0,133.0,0.0,*,test
4,2019/03/28,ﾏｲﾊﾞｽｹﾂﾄﾒｸﾞﾛﾀｲｺﾊﾞ,本人,1回払い,397.0,0.0,397.0,397.0,0.0,*,test


In [4]:
def extract_colname_and_dtype_from_dataframe(df):

    # 読み込んだデータのカラム名とデータ型を抽出
    dtypes_df = pd.DataFrame(df.dtypes).reset_index()
    dtypes_df.columns=['カラム名','データ型']

    return dtypes_df


def get_colname_and_dtype_to_create_table(df):
    '''
    関数内容
    ・create tableに必要なカラム名とデータ型を取得し、2つを特定の形式に並べた文字列を作成する関数
    '''
    # 読み込んだデータのカラム名とデータ型を抽出したデータフレームの作成
    df1 = extract_colname_and_dtype_from_dataframe(df)

    # SQLServerへのインポート用のデータ型名にリネーム
    df1['データ型'] = np.where(df1['データ型']=='object', 'NVARCHAR(25)',
                                 np.where(df1['データ型']=='float64', 'NUMERIC(25)',df1['データ型']))

    ct_col_list = df1['カラム名'].tolist()
    # カラム名に'/'が含まれているとSQLServerへのインポート時にエラーが発生するのでリネーム
    ct_col_list =['['+ col_name + ']' if '/' in col_name else col_name for col_name in ct_col_list]
    # カラム名先頭が数字の場合もSQLServerへのインポート時にエラーが発生するのでリネーム
    ct_col_list =['['+ col + ']' if re.match('^[0-9]', col) else col for col in ct_col_list ]
    
    ct_dtype_list = df1['データ型'].tolist()

    combi_ct_list = [str(col_name)+' '+str(dtype_name) for col_name, dtype_name in zip (ct_col_list, ct_dtype_list)]

    colname_and_dtype_str = ','.join(map(str, combi_ct_list))

    return colname_and_dtype_str

def get_colname_and_questionmark_to_insert_data(df):
    '''
    関数内容
    ・insert intoに必要なカラム名と?マークを取得する関数

    '''
    # 読み込んだデータのカラム名とデータ型を抽出したデータフレームの作成
    df1 = extract_colname_and_dtype_from_dataframe(df)

    it_col_list = df1['カラム名'].tolist()
    # カラム名に'/'が含まれているとSQLServerへのインポート時にエラーが発生するのでリネーム
    it_col_list =['['+ col_name + ']' if '/' in col_name else col_name for col_name in it_col_list]
    # カラム名先頭が数字の場合もSQLServerへのインポート時にエラーが発生するのでリネーム
    it_col_list =['['+ col + ']' if re.match('^[0-9]', col) else col for col in it_col_list ]

    it_col_str = ','.join(map(str, it_col_list))

    questionmark_str = '?,'*len(df1)
    # 最後の「,」は必要ない + ()を追加
    questionmark_str = '(' + questionmark_str[:-1] + ')'

    return it_col_str, questionmark_str

In [5]:
get_colname_and_dtype_to_create_table(df)

'利用日 NVARCHAR(25),利用店名・商品名 NVARCHAR(25),利用者 NVARCHAR(25),支払方法 NVARCHAR(25),利用金額 NUMERIC(25),支払手数料 NUMERIC(25),支払総額 NUMERIC(25),[4月支払金額] NUMERIC(25),[5月繰越残高] NUMERIC(25),新規サイン NVARCHAR(25),[test/test] NVARCHAR(25)'

In [6]:
get_colname_and_questionmark_to_insert_data(df)

('利用日,利用店名・商品名,利用者,支払方法,利用金額,支払手数料,支払総額,[4月支払金額],[5月繰越残高],新規サイン,[test/test]',
 '(?,?,?,?,?,?,?,?,?,?,?)')